In [0]:
%matplotlib inline
!pip install -U tables
import pandas as pd
import time
import numpy as np
import sklearn
from sklearn import linear_model
import json
import matplotlib.pyplot as plt
import os
from tqdm.notebook import tqdm
import mxnet as mx
import pandarallel
from mxboard import *

In [0]:
#from google.colab import drive
#drive.mount('/content/gdrive')
seed = 42
dataset_root = os.path.expanduser('~/TaxiData')

In [0]:
num_samples = 1000000
train_indices, test_indices = sklearn.model_selection.train_test_split(np.arange(num_samples))

def get_dataset():
    dataset = pd.read_hdf(os.path.join(dataset_root, 'data.h5'))
    train, test = sklearn.model_selection.train_test_split(dataset, random_state=seed)
    train = train.copy()
    test=  test.copy()
    return train, test

In [0]:
train, test = get_dataset()

In [0]:
use_numerical_cat = True

In [0]:
category_features = {
    'VendorID':(2, 2),
    'PULocationID': (265, 30),
    'DOLocationID': (265, 30),
    'payment_type': (5, 2),
    'week': (53, 10),
    'dayofweek': (7, 3),
    'quarterofday': (96, 20)
}
exclude = ['tpep_pickup_datetime', 'tpep_dropoff_datetime'] #+ list(category_features.keys())
columns = [c for c in train.columns if c not in exclude]
#columns += list(category_features.keys())

In [0]:
train_pickup_counts = {i: 0 for i in range(1, 266)}
train_dropoff_counts = {i: 0 for i in range(1, 266)}
train_pickup_counts.update(train['PULocationID'].value_counts().to_dict())
train_dropoff_counts.update(train['DOLocationID'].value_counts().to_dict())

def get_pickup_dropoff_count(pu_map, do_map):
    def func(x):
      return pu_map[x['PULocationID']], do_map[x['DOLocationID']]
    return func

In [0]:
def get_date(df):
    df['week'] = df['tpep_pickup_datetime'].dt.weekofyear
    df['dayofweek'] = df['tpep_pickup_datetime'].dt.dayofweek
    df['quarterofday'] = df['tpep_pickup_datetime'].dt.hour * 4 + df['tpep_pickup_datetime'].dt.quarter
    df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds()
    df.drop(columns=['tpep_dropoff_datetime', 'tpep_pickup_datetime'], inplace=True)

In [0]:
def haversine_array(lat1, lng1, lat2, lng2):
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    AVG_EARTH_RADIUS = 6371  # in km
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2
    h = 2 * AVG_EARTH_RADIUS * np.arcsin(np.sqrt(d))
    return h

def dummy_manhattan_distance(lat1, lng1, lat2, lng2):
    a = haversine_array(lat1, lng1, lat1, lng2)
    b = haversine_array(lat1, lng1, lat2, lng1)
    return a + b

def bearing_array(lat1, lng1, lat2, lng2):
    AVG_EARTH_RADIUS = 6371  # in km
    lng_delta_rad = np.radians(lng2 - lng1)
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    y = np.sin(lng_delta_rad) * np.cos(lat2)
    x = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(lng_delta_rad)
    return np.degrees(np.arctan2(y, x))

In [0]:
def preprocess(df, drop_outlier):
    get_date(df)
    df['direction'] = bearing_array(df['PULocationX'].values, df['PULocationY'].values, df['DOLocationX'].values, df['DOLocationY'].values)
    df['distance_haversine'] = haversine_array(df['PULocationX'].values, df['PULocationY'].values, df['DOLocationX'].values, df['DOLocationY'].values)
    df['distance_dummy_manhattan'] = dummy_manhattan_distance(df['PULocationX'].values, df['PULocationY'].values, df['DOLocationX'].values, df['DOLocationY'].values)
    if drop_outlier:
        df.drop(df.loc[df['duration'] > 8*3600].index, inplace=True) # drop > 8 hours
    #df['pickup_loc_count'], df['dropoff_loc_count'] = zip(*df.parallel_apply(get_pickup_dropoff_count(train_pickup_counts, train_dropoff_counts), axis=1))
    dtype = 'float32'
    duration = df['duration'].to_numpy(dtype=dtype)
    # cats = [df[category_feature].to_numpy(dtype=dtype) for category_feature in category_features.keys()]
    conds = df[columns].to_numpy(dtype=dtype)
    return conds, duration

In [0]:
train_data, train_labels = preprocess(train, True)
test_data, test_labels = preprocess(test, True)

In [0]:
y_range = train['duration'].max()
y_range

In [0]:
del train
del test

In [0]:
from sklearn.metrics import mean_absolute_error

In [0]:
def fit_and_test(model, X, y, metric=mean_absolute_error):
    '''
    Fit the model and test the performance on the training set
    '''
    model.fit(X, y)
    y_pred = model.predict(X)
    print(type(model))
    print(metric(y, y_pred))
    return model

def test(model, X_test, y_test, metric=mean_absolute_error):
    y_pred = model.predict(X_test)
    y_pred = np.maximum(y_pred, 1.)
    ret = metric(y_test, y_pred)
    # print(ret)
    return ret

In [0]:
linear_reg = fit_and_test(sklearn.linear_model.LinearRegression(), train_data, train_labels)

In [0]:
train_pred = linear_reg.predict(train_data)
test_pred = linear_reg.predict(test_data)
import pickle
pickle.dump(train_pred, open('train_pred_lr.pkl','wb'))
pickle.dump(test_pred, open('test_pred_lr.pkl','wb'))

In [0]:
print('mae', test(linear_reg, test_data, test_labels))
print('rmse', np.sqrt(test(linear_reg, test_data, test_labels, sklearn.metrics.mean_squared_error)))
print('rmsle', np.sqrt(test(linear_reg, test_data, test_labels, sklearn.metrics.mean_squared_log_error)))

In [0]:
np.sqrt(398986.7)

In [0]:
#import xgboost
#xgb_reg = fit_and_test(xgboost.XGBRegressor(), train_data, train_labels)

In [0]:
import scipy
(test_labels - train_labels.mean()).mean() * 3600

In [0]:
import sklearn.datasets
#sklearn.datasets.dump_svmlight_file(train_data, train_labels, './numerical/train.txt')
#sklearn.datasets.dump_svmlight_file(test_data, test_labels, './numerical/test.txt')

In [0]:
# onehot encoding

In [0]:
#onehot_proc = sklearn.preprocessing.OneHotEncoder(n_values=[in_dim for (in_dim, _) in category_features.values()])
#from sklearn.compose import ColumnTransformer
#ct = ColumnTransformer([('onehot', onehot_proc, list(range(len(columns) - len(category_features), len(columns))))], sparse_threshold=1.0, remainder='passthrough')
#train_data = ct.fit_transform(train_data)
#test_data = ct.fit_transform(test_data)

In [0]:
#import sklearn.datasets
#sklearn.datasets.dump_svmlight_file(train_data, train_labels, './onehot/train.txt')
#sklearn.datasets.dump_svmlight_file(test_data, test_labels, './onehot/test.txt')

In [0]:
import sklearn.ensemble
rfr = fit_and_test(sklearn.ensemble.RandomForestRegressor(n_estimators=32, n_jobs=1, max_depth=15), train_data, train_labels)

In [0]:
sklearn.__version__

In [0]:
print('mae', test(rfr, test_data, test_labels))
print('rmse', np.sqrt(test(rfr, test_data, test_labels, sklearn.metrics.mean_squared_error)))
print('rmsle', np.sqrt(test(rfr, test_data, test_labels, sklearn.metrics.mean_squared_log_error)))

In [0]:
print('mae', test(rfr, train_data, train_labels))
print('rmse', np.sqrt(test(rfr, train_data, train_labels, sklearn.metrics.mean_squared_error)))
print('rmsle', np.sqrt(test(rfr, train_data, train_labels, sklearn.metrics.mean_squared_log_error)))

In [0]:
train_pred = rfr.predict(train_data)
test_pred = rfr.predict(test_data)

In [0]:
#import pickle
#pickle.dump(rfr,open('random_forest_reg.pkl','wb')) 

In [0]:
import xgboost as xgb

model_path = './numerical/0099.model'
test_data_path = './xgb/test.txt'
dtrain = xgb.DMatrix(train_data)

In [0]:
import xgboost as xgb

In [0]:
bst = xgb.Booster(model_file='./numerical/0099.model')

In [0]:
test_pred = bst.predict(xgb.DMatrix('./numerical/test.txt#dest1.cache'))

In [0]:
np.sqrt(((test_labels - test_pred)**2).mean())

In [0]:
import pickle
pickle.dump(train_pred, open('xgb_train_pred.pkl', 'wb'))

In [0]:
f = open('xgb_train_pred.pkl', 'rb')
print(f)
X1 = pickle.load(f)
X2 = open('rfr_train_pred.pkl', 'rb')
X2 = pickle.load(X2)
X3 = open('xgb_test_pred.pkl','rb')
X3 = pickle.load(X3)
X4 = open('rfr_test_pred.pkl','rb')
X4 = pickle.load(X4)

In [0]:
X5 = pickle.load(open('train_pred_lr.pkl','rb'))
X6 = pickle.load(open('test_pred_lr.pkl','rb'))

In [0]:
X_tr1 = np.array([X1,X2,X5]).T

In [0]:
X_te1 = np.array([X3,X4,X6]).T

In [0]:
linear_b = fit_and_test(sklearn.linear_model.LinearRegression(), X_tr, train_labels)

In [0]:
test(linear_b, X_te, test_labels)

In [0]:
linear_b1 = fit_and_test(sklearn.linear_model.LinearRegression(), X_tr1, train_labels)

In [0]:
test(linear_b1,X_te1,test_labels)

In [0]:
np.sqrt(test(linear_b1,X_te1,test_labels,sklearn.metrics.mean_squared_error))